In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler


# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")

data = data.drop("date", axis=1)

data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()

data = data.astype('float32')


# Keep data until 31.08.2023
data = data.iloc[:10731]

#print(data['open'].dtype)
#print(data.shape)

# Assuming 'data' is a pandas DataFrame
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']]
y_data = data["close"]

# Now x_data and y_data are pandas DataFrames/Series, respectively

x_data.tail(1)




# Set the window size for training
train_window_size = 20

# Initialize lists to store training and temporary sets
x_train_list, y_train_list, x_temp_list, y_temp_list = [], [], [], []

# Iterate through the data with the specified window size
for i in range(0, len(x_data) - train_window_size, train_window_size + 1):
    x_train_temp = x_data.iloc[i:i+train_window_size+1]
    y_train_temp = y_data.iloc[i:i+train_window_size+1]

    # Separate the last row for the temporary set
    x_train = x_train_temp.iloc[:-1]
    y_train = y_train_temp.iloc[:-1]

    x_temp = x_train_temp.iloc[-1:]
    y_temp = y_train_temp.iloc[-1:]

    x_train_list.append(x_train)
    y_train_list.append(y_train)
    x_temp_list.append(x_temp)
    y_temp_list.append(y_temp)

# Concatenate the lists into pandas DataFrames
x_train = pd.concat(x_train_list)
y_train = pd.concat(y_train_list)
x_temp = pd.concat(x_temp_list)
y_temp = pd.concat(y_temp_list)

# print(y_train.head(50))
x_temp_train, x_temp_val, y_temp_train, y_temp_val = train_test_split(x_temp, y_temp, test_size=0.2, random_state=42)


# Split x_temp and y_temp into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.5, random_state=42)


"""
# Print the last 5 rows of x_data
print("Last 5 rows of x_data:")
print(x_data.tail(5))

# Print the last 5 rows of x_train
print("\nLast 25 rows of x_train:")
print(x_train.tail(25))

print("\nLast 3 rows of y_train:")
print(y_temp.tail(3))
"""




scaler = MinMaxScaler()

x_train_normalized = scaler.fit_transform(x_train)
x_val_normalized = scaler.transform(x_val)
x_test_normalized = scaler.transform(x_test)

# Convert the data to PyTorch tensors
x_train_tensor = torch.tensor(x_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

x_val_tensor = torch.tensor(x_val_normalized, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

x_test_tensor = torch.tensor(x_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)




"""
# x_train_tensor inverse

x_test_original = scaler.inverse_transform(x_train_tensor.numpy())
print("\nFirst row of x_test_original:")
print(x_test_original[0])

print("\nFirst row of x_train:")
print(x_train.head(1))



print("\nLast row of x_test_original:")
print(x_test_original[-1])

print("\nLast row of x_train:")
print(x_train.tail(1))
"""



'\n# x_train_tensor inverse\n\nx_test_original = scaler.inverse_transform(x_train_tensor.numpy())\nprint("\nFirst row of x_test_original:")\nprint(x_test_original[0])\n\nprint("\nFirst row of x_train:")\nprint(x_train.head(1))\n\n\n\nprint("\nLast row of x_test_original:")\nprint(x_test_original[-1])\n\nprint("\nLast row of x_train:")\nprint(x_train.tail(1))\n'

In [11]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, learning_rate=0.001, window_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50

window_size = 2

input_size = 7  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 4

learning_rate = 0.0001  # Learning rate
window_size = 1 # Window size
model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 22351.461821838264, Validation Loss: 22762.787109375
Epoch [2/50], Training Loss: 19530.393213252326, Validation Loss: 20687.96484375
Epoch [3/50], Training Loss: 16522.507419584123, Validation Loss: 19053.494140625
Epoch [4/50], Training Loss: 14229.592356875843, Validation Loss: 17442.82421875
Epoch [5/50], Training Loss: 12442.620746553148, Validation Loss: 14169.869140625
Epoch [6/50], Training Loss: 11001.347510759988, Validation Loss: 11881.30859375
Epoch [7/50], Training Loss: 9835.767752316897, Validation Loss: 10697.4609375
Epoch [8/50], Training Loss: 8900.346939763613, Validation Loss: 9903.060546875
Epoch [9/50], Training Loss: 8018.447919958429, Validation Loss: 12919.5048828125
Epoch [10/50], Training Loss: 7232.683605896568, Validation Loss: 11749.7705078125
Epoch [11/50], Training Loss: 6530.987028128451, Validation Loss: 11055.7822265625
Epoch [12/50], Training Loss: 5885.8008603381895, Validation Loss: 10155.583984375
Epoch [13/50], Traini

In [12]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, learning_rate=0.001, window_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50

window_size = 4

input_size = 7  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 4

learning_rate = 0.0001  # Learning rate
window_size = 1 # Window size
model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



Epoch [1/50], Training Loss: 22319.8968181098, Validation Loss: 22740.08984375
Epoch [2/50], Training Loss: 19575.82829728923, Validation Loss: 20715.751953125
Epoch [3/50], Training Loss: 16560.932099565358, Validation Loss: 19093.005859375
Epoch [4/50], Training Loss: 14259.677255592298, Validation Loss: 17747.6875
Epoch [5/50], Training Loss: 12471.883559377153, Validation Loss: 14551.69140625
Epoch [6/50], Training Loss: 11028.106706517407, Validation Loss: 12054.23046875
Epoch [7/50], Training Loss: 9862.15668895215, Validation Loss: 10690.443359375
Epoch [8/50], Training Loss: 8923.620887357007, Validation Loss: 9940.3759765625
Epoch [9/50], Training Loss: 8032.467596803317, Validation Loss: 12909.0341796875
Epoch [10/50], Training Loss: 7250.105078278011, Validation Loss: 11969.3408203125
Epoch [11/50], Training Loss: 6550.6564129727285, Validation Loss: 11309.580078125
Epoch [12/50], Training Loss: 5906.232188705084, Validation Loss: 10684.69140625
Epoch [13/50], Training Loss:

In [13]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, learning_rate=0.001, window_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50

window_size = 7

input_size = 7  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 4

learning_rate = 0.0001  # Learning rate
window_size = 1 # Window size
model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



Epoch [1/50], Training Loss: 22368.88652702448, Validation Loss: 22775.37109375
Epoch [2/50], Training Loss: 19616.243053061364, Validation Loss: 20743.466796875
Epoch [3/50], Training Loss: 16566.98257264973, Validation Loss: 19091.939453125
Epoch [4/50], Training Loss: 14270.991436727287, Validation Loss: 17720.076171875
Epoch [5/50], Training Loss: 12479.091879509158, Validation Loss: 14486.3017578125
Epoch [6/50], Training Loss: 11032.177518863627, Validation Loss: 12007.5517578125
Epoch [7/50], Training Loss: 9863.860659070362, Validation Loss: 10707.0263671875
Epoch [8/50], Training Loss: 8924.358268312853, Validation Loss: 9945.9072265625
Epoch [9/50], Training Loss: 8042.379863454362, Validation Loss: 13071.2744140625
Epoch [10/50], Training Loss: 7256.935003655125, Validation Loss: 12077.9697265625
Epoch [11/50], Training Loss: 6556.560700647632, Validation Loss: 11557.6591796875
Epoch [12/50], Training Loss: 5912.539038845866, Validation Loss: 10946.0390625
Epoch [13/50], Tra

In [14]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, learning_rate=0.001, window_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50

window_size = 10

input_size = 7  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 4

learning_rate = 0.0001  # Learning rate
window_size = 1 # Window size
model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



Epoch [1/50], Training Loss: 22317.428971511974, Validation Loss: 22738.310546875
Epoch [2/50], Training Loss: 19572.714614360568, Validation Loss: 20713.8046875
Epoch [3/50], Training Loss: 16554.31648854807, Validation Loss: 19095.16796875
Epoch [4/50], Training Loss: 14263.27642229091, Validation Loss: 17919.138671875
Epoch [5/50], Training Loss: 12482.496839357547, Validation Loss: 14918.21875
Epoch [6/50], Training Loss: 11040.810606993069, Validation Loss: 12291.623046875
Epoch [7/50], Training Loss: 9875.969814177493, Validation Loss: 10688.41015625
Epoch [8/50], Training Loss: 8935.643862821089, Validation Loss: 9922.2998046875
Epoch [9/50], Training Loss: 8057.432918739009, Validation Loss: 13169.2451171875
Epoch [10/50], Training Loss: 7269.159717453258, Validation Loss: 12144.662109375
Epoch [11/50], Training Loss: 6568.936487477045, Validation Loss: 11621.5478515625
Epoch [12/50], Training Loss: 5924.049974036686, Validation Loss: 11027.7607421875
Epoch [13/50], Training Lo

In [15]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, learning_rate=0.001, window_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50

window_size = 12

input_size = 7  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 4

learning_rate = 0.0001  # Learning rate
window_size = 1 # Window size
model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



Epoch [1/50], Training Loss: 22378.59537625562, Validation Loss: 22782.4140625
Epoch [2/50], Training Loss: 19631.858954196523, Validation Loss: 20753.775390625
Epoch [3/50], Training Loss: 16583.19557265353, Validation Loss: 19101.595703125
Epoch [4/50], Training Loss: 14284.49371309695, Validation Loss: 17785.65234375
Epoch [5/50], Training Loss: 12493.65131585586, Validation Loss: 14659.44140625
Epoch [6/50], Training Loss: 11046.250683409366, Validation Loss: 12117.2666015625
Epoch [7/50], Training Loss: 9876.927548880853, Validation Loss: 10698.1474609375
Epoch [8/50], Training Loss: 8935.102401780161, Validation Loss: 9948.486328125
Epoch [9/50], Training Loss: 8044.144197207675, Validation Loss: 12721.01953125
Epoch [10/50], Training Loss: 7259.599931496834, Validation Loss: 11837.876953125
Epoch [11/50], Training Loss: 6554.783877510416, Validation Loss: 11162.9189453125
Epoch [12/50], Training Loss: 5901.113314770404, Validation Loss: 10239.109375
Epoch [13/50], Training Loss:

In [16]:
ä

NameError: name 'ä' is not defined

In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, learning_rate=0.001, window_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50

window_size = 1

input_size = 7  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 4

learning_rate = 0.0001  # Learning rate
window_size = 1 # Window size
model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, learning_rate=0.001, window_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50

window_size = 1

input_size = 7  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 4

learning_rate = 0.0001  # Learning rate
window_size = 1 # Window size
model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Define the size of the sliding window
# sliding_window_size = 1

# Walk-forward validation training with sliding window
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using sliding window approach
    for i in range(len(x_train_tensor)):
        # Determine the end index of the current window
        window_end = min(i + window_size, len(x_train_tensor))

        # Extract the current window of data
        inputs = x_train_tensor[i:window_end].unsqueeze(0)  # Add extra dimensions for batch and sequence length
        labels = y_train_tensor[window_end - 1]  # Label is the last value in the window

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            # Determine the end index of the current window
            window_end = min(i + window_size, len(x_val_tensor))

            inputs = x_val_tensor[i:window_end].unsqueeze(0)
            labels = y_val_tensor[window_end - 1]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')



c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 22362.390420705764, Validation Loss: 22770.517578125
Epoch [2/50], Training Loss: 19625.774810691677, Validation Loss: 20748.962890625
Epoch [3/50], Training Loss: 16572.38601870388, Validation Loss: 19089.185546875
Epoch [4/50], Training Loss: 14275.918697014538, Validation Loss: 17655.80078125
Epoch [5/50], Training Loss: 12485.246166992663, Validation Loss: 14549.3955078125
Epoch [6/50], Training Loss: 11038.983618900053, Validation Loss: 12068.533203125
Epoch [7/50], Training Loss: 9869.84869106432, Validation Loss: 10690.11328125
Epoch [8/50], Training Loss: 8928.599015030866, Validation Loss: 9931.947265625
Epoch [9/50], Training Loss: 8037.044813161406, Validation Loss: 12673.654296875
Epoch [10/50], Training Loss: 7255.179926998209, Validation Loss: 12002.94140625
Epoch [11/50], Training Loss: 6556.284719373862, Validation Loss: 11445.2900390625
Epoch [12/50], Training Loss: 5911.89182833453, Validation Loss: 10921.658203125
Epoch [13/50], Training 

In [ ]:
ä

NameError: name 'ä' is not defined

num_layers = 16 to slow

In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, learning_rate=0.001, window_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50

input_size = 7  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 16
# Number of LSTM layers
learning_rate = 0.0001  # Learning rate
window_size = 20  # Window size
model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)




# Walk-forward validation training
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using x_train_tensor and y_train_tensor
    for i in range(len(x_train_tensor)):
        inputs = x_train_tensor[i].unsqueeze(0).unsqueeze(0)  # Add two extra dimensions for batch and sequence length
        labels = y_train_tensor[i]

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            inputs = x_val_tensor[i].unsqueeze(0).unsqueeze(0)
            labels = y_val_tensor[i]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

# After training, you can test the model similarly using x_test_tensor and y_test_tensor


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 22327.921713422566, Validation Loss: 22746.3828125
Epoch [2/50], Training Loss: 19808.118937096144, Validation Loss: 20861.662109375
Epoch [3/50], Training Loss: 18020.592938645852, Validation Loss: 19660.25390625
Epoch [4/50], Training Loss: 15919.224945660948, Validation Loss: 18331.97265625
Epoch [5/50], Training Loss: 13533.551605080094, Validation Loss: 17175.2890625
Epoch [6/50], Training Loss: 12019.459947555484, Validation Loss: 17480.8515625
Epoch [7/50], Training Loss: 10777.384772162273, Validation Loss: 19364.748046875
Epoch [8/50], Training Loss: 9817.751082037768, Validation Loss: 19100.451171875
Epoch [9/50], Training Loss: 9099.111255824613, Validation Loss: 19494.08203125
Epoch [10/50], Training Loss: 8383.335022393132, Validation Loss: 20797.99609375
Epoch [11/50], Training Loss: 7790.24467633839, Validation Loss: 21700.71484375
Epoch [12/50], Training Loss: 8141.076041850022, Validation Loss: 28721.216796875
Epoch [13/50], Training Loss: 

KeyboardInterrupt: 

In [ ]:
ä

In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1, learning_rate=0.001, window_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.window_size = window_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        self.learning_rate = learning_rate

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        return out

# Initialize the model with specified parameters

num_epochs = 50

input_size = 7  # Number of features
hidden_size = 64  # Number of LSTM units
num_layers = 4
# Number of LSTM layers
learning_rate = 0.0001  # Learning rate
window_size = 20  # Window size
model = LSTMModel(input_size, hidden_size, num_layers, learning_rate, window_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)




# Walk-forward validation training
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    # Train the model using x_train_tensor and y_train_tensor
    for i in range(len(x_train_tensor)):
        inputs = x_train_tensor[i].unsqueeze(0).unsqueeze(0)  # Add two extra dimensions for batch and sequence length
        labels = y_train_tensor[i]

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validate the model after each epoch using x_val_tensor and y_val_tensor
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
        for i in range(len(x_val_tensor)):
            inputs = x_val_tensor[i].unsqueeze(0).unsqueeze(0)
            labels = y_val_tensor[i]

            outputs = model(inputs)
            val_loss += criterion(outputs, labels)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {running_loss / len(x_train_tensor)}, Validation Loss: {val_loss / len(x_val_tensor)}')

# After training, you can test the model similarly using x_test_tensor and y_test_tensor


c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Training Loss: 22360.31667383806, Validation Loss: 22769.455078125
Epoch [2/50], Training Loss: 19505.713305504094, Validation Loss: 20672.65234375
Epoch [3/50], Training Loss: 16501.247543511898, Validation Loss: 19049.033203125
Epoch [4/50], Training Loss: 14218.50668195493, Validation Loss: 17513.68359375
Epoch [5/50], Training Loss: 12438.109628592638, Validation Loss: 14365.9365234375
Epoch [6/50], Training Loss: 11001.18178465093, Validation Loss: 11959.9912109375
Epoch [7/50], Training Loss: 9838.843309614864, Validation Loss: 10677.056640625
Epoch [8/50], Training Loss: 8904.14371472347, Validation Loss: 9903.6171875
Epoch [9/50], Training Loss: 8020.264743066753, Validation Loss: 12834.7744140625
Epoch [10/50], Training Loss: 7236.501935498802, Validation Loss: 11874.6171875
Epoch [11/50], Training Loss: 6536.458665778439, Validation Loss: 11239.4130859375
Epoch [12/50], Training Loss: 5891.431617533891, Validation Loss: 10446.9951171875
Epoch [13/50], Training L